### 1.2 过程和它所产生的计算过程

上一节学习了基本算数运算符，将它们进行组合，并通过抽象方法将其定义为组合过程。然而就像是学习了象棋中的基本规则一样，我们对于典型的开局、策略等还没有经验，能够准确预料不同过程所产生的计算过程及结果是成为专家的关键。

#### 1.2.1 线性递归和迭代

考虑阶乘的计算：

$$n! = n*(n-1)*(n-2)...3*2*1$$

其中一种计算方法是：

$$n! = n*(n-1)!$$

In [1]:
(define (factorial n)
  (if (= n 1)
      1
      (* n (factorial (- n 1)))))
(factorial 6)

720

利用1.1.5中提到的替换模型，我们可以观察`(fractorial 6)`的执行过程如下图所示：

![figure 1.3](imgs/figure-1.3.png)

另一种计算方法，从1到n依次相乘，记录每次相乘的乘积：

In [2]:
(define (factorial n)
  (fact-iter 1 1 n))

(define (fact-iter prod counter max-count)
  (if (> counter max-count)
      prod
      (fact-iter (* counter prod)
                 (+ counter 1)
                 max-count)))

(factorial 6)

720

利用替换模型，再次观察`(fractorial 6)`的执行过程如下图所示：

![figure 1.4](imgs/figure-1.4.png)

---

比较两种计算方法，第一种方法称为**递归计算过程（recursive process）**，先通过扩展形成一条推迟计算的链条，完全展开后再逐步缩短完成真正的计算过程。完成这一计算过程需要解释器保存链条前段被推迟的计算。以`n!`的计算为例，解释器所需要保存的信息与`n`成正比，因此这类计算过程又称为**线性递归计算过程（linear recursive process）**。

第二种方法没有用到先扩展后收缩的方式，而是通过记录每一步运算中`n`、`max-count`、`counter`、`prod`的值，这种方法称为**迭代计算过程（iterative process）**，迭代计算过程利用几个**状态变量（state variables）**和一些规则来描述不同状态之间变化、更新的过程，并决定计算过程停止的条件。`n!`所需的迭代步骤随着`n`的增长线性增加，因此这类计算过程也称为**线性迭代计算过程（linear iterative process）**。

---

这里必须明确区分**计算过程 process**与**过程 procedure**的差别，procedure是指语法形式上的，而process是指计算过程的演化；这两者的差异容易让人弄混的地方在于，在一般语言的实现中，递归计算过程所消耗的内存资源随着过程调用的增加而增加，……，After Ch5 Here。

#### 